In [3]:
import numpy as np

# --- 1. SETUP: Define images and kernel ---

# Define 2x2 kernel for correlation
h = np.array([[-1, 1], [-1, 1]])

# Create 10x10 image of zeros.
f = np.zeros((10, 10))
## Insert 5x5 block of ones in center.
f[3:8, 3:8] = np.ones((5, 5))

# We correlate with (1 - f), which flips the 0s and 1s.
image_to_process = 1 - f

print("--- Input Image (1 - f) ---")
print(image_to_process, "\n")


# --- 2. BOUNDARY HANDLING FUNCTIONS ---

def my_array_ref(a, i, j):  # Return pixel value or 0 if out-of-bounds
    """
    Checks if a pixel is inside the image.
    If it is, returns the pixel's value.
    If it's out of bounds, returns 0 (Zero-Padding).
    """
    if 0 <= i < a.shape[0] and 0 <= j < a.shape[1]:
        return a[i, j]
    else:
        return 0

def my_array_ref_bound(a, i, j):
  # Clamp out-of-bounds to nearest edge
    """
    Checks if a pixel is inside the image.
    If it's out of bounds, it "clamps" the coordinates
    to the nearest edge pixel instead of returning 0 (Boundary Clamping).
    """
    # If the spot is off the top/bottom, use the edge spot instead.
    if i < 0:
        i = 0
    elif i >= a.shape[0]:
        i = a.shape[0] - 1

    # If the spot is off the left/right, use the edge spot instead.
    if j < 0:
        j = 0
    elif j >= a.shape[1]:
        j = a.shape[1] - 1

    return a[i, j]


# --- 3. CORRELATION CALCULATIONS ---

def compute_corr(image, kernel, boundary_func):
  # Compute 2D correlation with given boundary handler
    """
    A general function to compute 2D correlation.
    It takes an image, a kernel, and a boundary-handling function as input.
    """
    # Create an empty map (all zeros) to store our results.
    output = np.zeros_like(image, dtype=float)

    # Go through each pixel of the image one by one.
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            prod_sum = 0.0
            # At each pixel, place our kernel and sum the products.
            for k in range(kernel.shape[0]):
                for l in range(kernel.shape[1]):
                    prod_sum += kernel[k, l] * boundary_func(image, i + k, j + l)
            output[i, j] = prod_sum
    return output


# --- 4. EXECUTE AND PRINT RESULTS ---

# Correlation with zero-padding
g_zero_padding = compute_corr(image_to_process, h, my_array_ref)
print("--- Output with Zero-Padding (Step 1) ---")
print(g_zero_padding, "\n")

# Correlation with boundary clamping
g_boundary_clamping = compute_corr(image_to_process, h, my_array_ref_bound)
print("--- Output with Boundary Clamping (Step 3) ---")
print(g_boundary_clamping)

--- Input Image (1 - f) ---
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]] 

--- Output with Zero-Padding (Step 1) ---
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0. -2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -2.]
 [ 0.  0. -1.  0.  0.  0.  0.  1.  0. -2.]
 [ 0.  0. -2.  0.  0.  0.  0.  2.  0. -2.]
 [ 0.  0. -2.  0.  0.  0.  0.  2.  0. -2.]
 [ 0.  0. -2.  0.  0.  0.  0.  2.  0. -2.]
 [ 0.  0. -2.  0.  0.  0.  0.  2.  0. -2.]
 [ 0.  0. -1.  0.  0.  0.  0.  1.  0. -2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -2.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]] 

--- Output with Boundary Clamping (Step 3) ---
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  

In [7]:
import numpy as np
import matplotlib.pyplot as plt

# 1D Gaussian function
def Gaussian(x, s): return np.exp(-x**2/(2*s**2))/(np.sqrt(2*np.pi)*s)
# Second derivative of Gaussian
def second_d_guassian(x, s): return ((x**2-s**2)/s**4)*G(x,s)

# Find best σ1, σ2 for DoG approx
def best_DoG(x, s_target, s1_range, s2_range):
    target = second_d_guassian(x, s_target); norm_t = np.linalg.norm(target)# Normalize for relative error
    best, err_best = None, np.inf # Track best match and lowest error
    for s1 in s1_range:
        for s2 in s2_range:
            if s2 <= s1: continue
            # Difference of Gaussians
            dog = Guassian(x,s1)-Guassian(x,s2)
            alpha = np.dot(dog,target)/np.dot(dog,dog)
            # Compute relative error
            err = np.linalg.norm(alpha*dog-target)/norm_t
             # Update best result if error improves
            if err < err_best:
                err_best, best = err, (s1,s2,alpha,alpha*dog,target)
    return best

# Parameters
# x-axis range
x = np.linspace(-50,50,2001)
s1,s2,alpha,approx,target = best_DoG(x,3,np.linspace(0.5,5,50),np.linspace(0.6,6,60))

# Results
# Print best parameters found
print(f"Best σ1 = {s1:.3f}, σ2 = {s2:.3f}")
plt.plot(x,target,label="Second derivative (σ=3)")
plt.plot(x,approx,'--',label=f"DoG (σ1={s1:.2f}, σ2={s2:.2f})")
plt.xlabel("x"); plt.ylabel("Amplitude"); plt.grid(); plt.legend()
plt.title("DoG Approximation of the Gaussian's Second Derivative")
plt.show()

NameError: name 'Guassian' is not defined